In [1]:
!pip install scattertext
!pip install spacy

In [27]:
import scattertext as st
import pandas as pd
import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [28]:
df = pd.read_csv('/Users/Link/Documents/5005数据可视化/Group project/archive/kindle_reviews.csv')
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


**Columns:**  
asin - 书籍id  
helpful - 有帮助的概率    
overall - 书籍评分  
reviewText - 评论文本  
reviewTime - 评论时间   
reviewerID - 评论者id   
reviewerName - 评论者名称  
summary - 评论综述  
unixReviewTime - 时间戳

In [29]:
# 定义函数来根据分数评价积极或消极
def get_evaluation(score):
    if score >= 4:
        return 'pos'
    elif score<=3:
        return 'neg'

# 将函数应用到每一行的分数列中，并将结果存储到评价列中
df['sentiment'] = df['overall'].apply(get_evaluation)

df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,sentiment
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000,pos
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400,pos
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600,pos
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000,pos
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200,pos


In [30]:
#test = df.loc[df['overall'] == 5]

In [31]:
review_text = df[['reviewText','sentiment']]

In [32]:
review_text

,reviewText,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,pos
1,This book is a reissue of an old one; the auth...,pos
2,This was a fairly interesting read. It had ol...,pos
3,I'd never read any of the Amy Brewster mysteri...,pos
4,"If you like period pieces - clothing, lingo, y...",pos
...,...,...
982614,Yasss hunny! This is a great read. That Dre is...,pos
982615,I ENJOYED THIS BOOK FROM BEGINNING TO END NOW ...,pos
982616,Great book! Cherika was a fool. She let that m...,pos
982617,When I say this was an excellent book please b...,pos


In [33]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [34]:
review_text_copy = review_text.copy()
review_text_copy['reviewText'] = review_text_copy['reviewText'].astype(str)

In [35]:
review_text_5000 = review_text_copy.iloc[:5000,:]

常见的预处理操作：
分词（tokenization）：将文本分成单独的单词或标记（token）。  
去除停用词（stop words removal）：去除一些常见但无意义的词语（如 "the"、"a"、"an" 等）。  
词干化（stemming）和词形还原（lemmatization）：将单词的不同形式（如复数形式、时态等）归一化为一个基本形式。  
去除标点符号（punctuation removal）：去除文本中的标点符号，这些符号通常不会提供太多的信息。  
去除 HTML 标签（HTML tag removal）：如果文本包含 HTML 标记，则需要去除它们，以便更好地处理文本。  
去除特殊字符（special character removal）：如果文本包含一些特殊字符（如 URL、邮件地址等），可以考虑去除它们。  

In [36]:
review_text_5000.head()

,reviewText,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,pos
1,This book is a reissue of an old one; the auth...,pos
2,This was a fairly interesting read. It had ol...,pos
3,I'd never read any of the Amy Brewster mysteri...,pos
4,"If you like period pieces - clothing, lingo, y...",pos


In [37]:
import re, string

# 将文本转化为小写
def lower_text(text):
    return text.lower()

# 去除文本中的标点符号
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# 去除换行符
def remove_newlines(text):
    return text.replace('\n', '')

# 去除文本中的stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    return " ".join([word for word in words if word.lower() not in stop_words])

nltk.download('punkt')
# 创建词干化器
stemmer = SnowballStemmer('english')

# 文本词干化和词形还原
def stem_and_lemma(text):
    text = text.lower()

    # 词形还原
    words = nltk.word_tokenize(text)

    # 词干化
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)


review_text_5000['reviewText'] = review_text_5000['reviewText'].apply(lower_text)\
                                                   .apply(remove_punctuation)\
                                                   .apply(remove_newlines)\
                                                   .apply(remove_stopwords)\
                                                   .apply(stem_and_lemma)\
                                                  
review_text_5000.head(10)

[nltk_data] Downloading package stopwords to /Users/Link/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Link/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/var/folders/qm/jh9yg7h129q058z58zlc62v80000gn/T/ipykernel_8913/309058347.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_text_5000['reviewText'] = review_text_5000['reviewText'].apply(lower_text)\


,reviewText,sentiment
0,enjoy vintag book movi enjoy read book plot un...,pos
1,book reissu old one author born 1910 era say n...,pos
2,fair interest read old style terminologyi glad...,pos
3,id never read ami brewster mysteri one realli ...,pos
4,like period piec cloth lingo enjoy mysteri aut...,pos
5,beauti indepth charact descript make like fast...,pos
6,enjoy one tho im sure call ami brewster myster...,pos
7,never heard ami brewster dont need like ami br...,pos
8,darth maul work cloak dark commit sabotag stor...,pos
9,short stori focus darth maul role help trade f...,pos


原text preprocessing弃用：

In [38]:
# # 下载停用词列表和词干化器
# nltk.download('stopwords')
# nltk.download('punkt')

# # 创建停用词列表
# stop_words = stopwords.words('english')

# # 创建词干化器
# stemmer = SnowballStemmer('english')

# # 定义预处理函数
# def preprocess(text):
    
#     # 将文本转换为小写
#     text = text.lower()

#     # 分词
#     words = nltk.word_tokenize(text)

#     # 去除停用词
#     words = [word for word in words if word not in stop_words]

#     # 词干化
#     words = [stemmer.stem(word) for word in words]

#     # 返回预处理后的文本
#     return ' '.join(words)

# # 对每行进行预处理
# review_text_5000['processed_text'] = review_text_5000['reviewText'].apply(preprocess)

# # 打印预处理后的结果
# print(review_text_5000['processed_text'])


In [39]:
review_text_5000

,reviewText,sentiment
0,enjoy vintag book movi enjoy read book plot un...,pos
1,book reissu old one author born 1910 era say n...,pos
2,fair interest read old style terminologyi glad...,pos
3,id never read ami brewster mysteri one realli ...,pos
4,like period piec cloth lingo enjoy mysteri aut...,pos
...,...,...
4995,good stuffthi terrif storyth write skill seen ...,pos
4996,short enough read chili cook keep interest beg...,pos
4997,end expect done didnt expect much 99cent book ...,neg
4998,say mix feel short although initiali start int...,neg


In [40]:
nlp = spacy.load('en_core_web_sm')
# 创建语料库
corpus = st.CorpusFromPandas(review_text_5000, category_col='sentiment', text_col='reviewText', nlp=nlp).build()


In [41]:
corpus.get_df()

,level_0,index,reviewText,sentiment
0,0,0,enjoy vintag book movi enjoy read book plot un...,pos
1,1,1,book reissu old one author born 1910 era say n...,pos
2,2,2,fair interest read old style terminologyi glad...,pos
3,3,3,id never read ami brewster mysteri one realli ...,pos
4,4,4,like period piec cloth lingo enjoy mysteri aut...,pos
...,...,...,...,...
4995,4995,4995,good stuffthi terrif storyth write skill seen ...,pos
4996,4996,4996,short enough read chili cook keep interest beg...,pos
4997,4997,4997,end expect done didnt expect much 99cent book ...,neg
4998,4998,4998,say mix feel short although initiali start int...,neg


In [42]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['pos_score'] = corpus.get_scaled_f_scores('pos')
pos_most = list(term_freq_df.sort_values(by='pos_score',ascending=False).index[:100])

In [43]:
term_freq_df['neg_score'] = corpus.get_scaled_f_scores('neg')
neg_most = list(term_freq_df.sort_values(by='neg_score',ascending=False).index[:100])

In [44]:
term_freq_df

,pos freq,neg freq,pos_score,neg_score
term,,,,
enjoy,1223,249,0.937685,0.062315
vintag,4,0,0.561051,0.438949
book,4205,1515,0.162445,0.837555
movi,91,24,0.916006,0.083994
read,3628,1007,0.911044,0.088956
...,...,...,...,...
statement place,1,0,0.503628,0.496372
place want,1,0,0.503628,0.496372
back fun,1,0,0.503628,0.496372


In [45]:
print('Positive words:',pos_most)
print('Negative words:',neg_most)

Positive words: ['morgan', 'nt wait', 'bella', 'joy', 'high recommend', 'must read', 'great read', 'zsadist', 'joy review', 'wait read', 'delight', 'nt put', 'love charact', 'cat', 'love book', 'book love', 'lisa', 'excel', 'love love', 'jackson', 'well worth', 'amaz', 'jon', 'thorough enjoy', 'brenda', 'vampir', 'angel', 'shane', 'emma', 'puma', 'captiv', 'rafe', 'grow', 'pride', 'enjoy read', 'realli enjoy', 'love read', 'one favorit', 'great book', 'awesom', 'fantast', 'put book', 'insid', 'cole', 'truli', 'map', 'luke', 'thorough', 'fairi tale', 'brian', 'brett', 'ms', 'bell', 'touch', 'favorit', 'daughter', 'fairi', 'love', 'visit', 'fascin', 'perfect', 'secret', 'classic', 'book great', 'could put', 'laugh', 'great stori', 'jule', 'bodi', 'home', 'max', 'blood', 'look forward', 'start read', 'brought', 'paranorm', 'aislinn', 'what', 'memori', 'known', 'pleasur', 'read next', 'passion', 'julia', 'happi', 'great', 'hall', 'collect', 'sensual', 'happili', 'fun read', 'ami', 'wait ne

In [46]:
html = st.produce_scattertext_explorer(corpus, category='pos', category_name='pos', not_category_name='neg',
                                      width_in_pixels=1000,metadata=review_text_5000['sentiment'])
open("scattertext_01.html",'wb').write(html.encode('utf-8'))


3150241

In [64]:
# from IPython.display import IFrame
# html_path = '/Users/Link/Documents/5005数据可视化/Group project/archive/scattertext_01.html'
# iframe = IFrame(src=html_path, width=1300, height=700)
# display(iframe)

In [65]:
review_text_5000 = review_text_5000.iloc[:5000,:]

In [66]:
nlp = spacy.load('en_core_web_sm')
# 创建语料库
corpus_5000 = st.CorpusFromPandas(review_text_5000, category_col='sentiment', text_col='reviewText', nlp=nlp).build().get_unigram_corpus().compact(st.ClassPercentageCompactor(term_count=2,term_ranker=st.OncePerDocFrequencyRanker))


In [67]:
html = st.produce_characteristic_explorer(
    corpus_5000,
    category='pos',
    category_name='pos',
    not_category_name='neg',
    metadata=review_text_5000['sentiment']
)
open('demo_characteristic_chart.html', 'wb').write(html.encode('utf-8'))

3070908

In [69]:
pip install empath

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for empath: filename=empath-0.89-py3-none-any.whl size=57799 sha256=243f15c555a43ac998fce7596a910e4f95ae579f69b270c287db37c6f033899b
  Stored in directory: /Users/Link/Library/Caches/pip/wheels/bc/12/66/f34f90f8d11dd984ba54f3d58fa578208caa85fa001d4d3c86
Successfully built empath
Note: you may need to restart the kernel to use updated packages.


In [70]:
import empath

feat_builder = st.FeatsFromOnlyEmpath()
empath_corpus = st.CorpusFromParsedDocuments(review_text_5000,
                                            category_col='sentiment',
                                            feats_from_spacy_doc=feat_builder,
                                             parsed_col='reviewText').build()
html = st.produce_scattertext_explorer(empath_corpus,
                                       category='pos',
                                     category_name='pos',
                                    not_category_name='neg',
                                      width_in_pixels=1000,
                                      metadata=review_text_5000['sentiment'],
                                   use_non_text_features=True,
                                      use_full_doc=True,
                                        topic_model_term_lists=feat_builder.get_top_model_term_lists())
open("Convention-Visualization-Empath.html", 'wb').write(html.encode('utf-8'))

4105140